In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 18 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

20 February 2023 ; 19:30:20


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,26,0,9.4,1.6,3.5,0.457,...,3.22,114.51,111.37,3.14,3.11,114.52,111.48,3.05,1,1
1,Bobby Portis,PF,27,MIL,47,14,26.8,6.0,12.0,0.499,...,3.22,114.51,111.37,3.14,3.11,114.52,111.48,3.05,0,1
6,Joe Ingles,SF,35,MIL,25,0,22.5,2.1,5.6,0.379,...,3.22,114.51,111.37,3.14,3.11,114.52,111.48,3.05,0,1
8,Khris Middleton,SF,31,MIL,17,7,21.4,4.9,11.6,0.421,...,3.22,114.51,111.37,3.14,3.11,114.52,111.48,3.05,0,1
9,MarJon Beauchamp,SF,22,MIL,38,9,14.8,2.1,5.3,0.398,...,3.22,114.51,111.37,3.14,3.11,114.52,111.48,3.05,1,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Malcolm Brogdon,25.3,50,0,BOS,18.4,2.460000,6.888000,14.7,1,2.6,4.428000,0.291256
1,Christian Wood,27.7,48,17,DAL,21.1,2.220833,6.662500,17.8,6,2.3,3.758333,0.273444
2,Norman Powell,25.8,50,7,LAC,18.1,1.640000,5.576000,17.0,4,1.7,3.444000,0.210749
3,Bobby Portis,26.8,47,14,MIL,18.6,1.744681,6.804255,14.4,2,1.6,3.314894,0.151005
4,Grant Williams,27.9,58,21,BOS,10.7,0.706897,5.089655,8.7,1,-1.5,2.403448,0.087271


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)